# Introduction to ConvNets

In [1]:
import plaidml.keras
plaidml.keras.install_backend()

In [2]:
from keras import models
from keras import layers

In [3]:
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation='relu',input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))

INFO:plaidml:Opening device "llvm_cpu.0"


A convnet takes input as tensors of `(img_height, img_width, img_channels)` in this case the size is (28,28,1) i.e. 1 channel only(the format of a MNIST image)

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


The output of every layer od `Conv2D` and `MaxPooling` is a 3D tensor. Width and height shrink as we go deeper into the layer. Next step is to feed the last output to a `Dense` layer(s). To do so:

In [5]:
model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

See that the `(3,3,64)` became `(576,)` before going to the `Dense` layers. And the last 10 softmax is because we are doing a 10 class classification. Now import data.

In [7]:
from keras.datasets import mnist
from keras.utils import to_categorical
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [8]:
train_images = train_images.reshape((60000,28,28,1))
train_images = train_images.astype('float32')/255
x_val = train_images[:10000]
train_images = train_images[10000:]

test_images = test_images.reshape((10000,28,28,1))
test_images = test_images.astype('float32')/255

train_labels = to_categorical(train_labels)
y_val = train_labels[:10000]

train_labels = train_labels[10000:]
test_labels = to_categorical(test_labels)

In [9]:
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [10]:
history = model.fit(train_images,train_labels
                    ,epochs=3
                    ,batch_size=64,
                   validation_data=(x_val,y_val))

Train on 50000 samples, validate on 10000 samples
Epoch 1/3
 6272/50000 [==>...........................] - ETA: 3:39 - loss: 0.7676 - acc: 0.7549

KeyboardInterrupt: 

In [ ]:
HIST = history.history
HIST.keys()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(HIST['loss'],'Dr--')
plt.plot(HIST['val_loss'],'Db--')
plt.plot(HIST['acc'],'Hr--')
plt.plot(HIST['val_acc'],'Hb--')
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_images,test_labels)

In [ ]:
test_acc, test_loss

The fundamental difference btw dense and conv layers is:
1. Dense layers learn global patterns in their input feature space.
2. Whereas conv layers learn local patterns like small bends or twists in images.

This give convnets two interesting properties:
1. The patterns they learn are translation invariant: after learning the pattern in the lower right corner of the picture the convnet can recognise it anywhere.
2. They can learn spatial hierarchies of patterns i.e. patterns of patterns.

Convolutions operate over 3D tensors called `feature maps` with 2 spatial axes as well as `depth/channels`. The convolution operation extracts patches from the input feature map and applies the same transformation to each feature to produce an output feature map, another 3D tensor. The different depths in the output tensor stands for `filters`. Filters encode specific aspects of the input data. 

In the above example : the input feature map is `(28,28,1)` and the output is `(26,26,32)` : it computes 32 filters. Each of the 32 filters contain a 26x26 grid which is a `response map` of filter over the input. 

Convolutions are defined by two key parameters : 
1. size of the patch extracted from the input. typically 3x3 or 5x5.
2. Depth of output feature map. The number of filters computed by the convolution. 

The arguments are passed as : `Conv2D(output_depth,(window_height,window_width))`

Convolution works by sliding this window of the specified size over every possible location and extracting a 3D patch of features. Each such 3D patch is then transformed via a tensor product with the same learned weight matric called `convolution kernel` into a 1D vector shape. These are then reassembled into a 3D map. The output width and height may differ from the input width and height due to : 
1. Border effects: the 3x3 filter grid cannot be centered around all of the points in s nxn grid. Obv some border elements will be left out. We can solve thr problem by `padding` : padding adds extra boundary elements to the input tensor. `Conv2D` takes the argument `padding` which when set to `valid` does nothing and when set to `same` adds boundary elements. 
2. Use of strides: the distance between two successive windows is called `stride` . It is possible to have strided convolutions i.e. with gaps between successive windows. 

To downsample featuremaps instead of strides we use `MaxPooling`. The first maxpooling operation converts the feature map of 26x26 to 13x13. MaxPooling consists of extracting windows from the input feature map and outputting the max value of each channel. MaxPooling is generally done with `2x2` windows and stride 2. 

What will happed if we remove `MAxPooling2D` from our model:
1. The model will not be conductive to learn high level patterns. 
2. The final output map will need a million parameters to be flattened.

One cal also use `AveragePooling2D` etc. 

# On a small dataset

PLAN: Construct a general convnet with low accuracy. Then use `data augmentation` to increase accuracy. Also use `feature extraction with  a pertained network` and `fine tuning a pretrained network`. Together these 3 strategies will give max acc possible. The data to be used is from the **dogs-versus-cats Kaggle** competetion and is downloaded in the same folder.

## Data gathering

In [ ]:
import os, shutil

In [ ]:
original_dataset_dir = '/home/sironton/MYSPACE/p/TF/dogs-vs-cats/train'
base_dir = '/home/sironton/MYSPACE/p/TF/cats_and_dogs_small'
os.mkdir(base_dir)

In [ ]:
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

In [ ]:
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

In [ ]:
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)

In [ ]:
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))
print('total test cat images:', len(os.listdir(test_cats_dir)))
print('total test dog images:', len(os.listdir(test_dogs_dir)))

## Modeling

In [30]:
from keras import layers
from keras import models

In [31]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 128)       147584    
__________

For the compilation step, you’ll go with the RMSprop optimizer, as usual. Because you ended the network with a single sigmoid unit, you’ll use binary crossentropy as the loss.

In [33]:
from keras import optimizers
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

## Data processing
The .jpeg files should be converted to floating point tensors with values from 0 to 1. 

In [34]:
from keras.preprocessing.image import ImageDataGenerator

In [35]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [41]:
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(150,150), batch_size=20,
    class_mode='binary')
validation_generator = test_datagen.flow_from_directory(
    validation_dir, target_size=(150,150),batch_size=20,
    class_mode='binary')

Found 2000 images belonging to 2 classes.


NameError: name 'validation_dir' is not defined

### GENERATOR:
A Python generator is an object that acts as an `iterator`: it’s an object you can use with the `for ... in` operator. Generators are built using the yield operator. Here is an example of a generator that yields integers:

In [ ]:
def generator():
    i = 0
    while True:
        i += 1
        yield i
for item in generator():
    print(item)
    if item > 4:
        break

In [ ]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

Let’s fit the model to the data using the generator. You do so using the `fit_generator` method, the equivalent of `fit` for data generators. Because the data is being generated endlessly, the Keras model needs to know how many samples to draw from the generator before declaring an epoch over. This is the role of the `steps_per_epoch` argument: after having drawn steps_per_epoch batches from the generator the fitting process will go to the next epoch. When using `fit_generator` , you can pass a `validation_data` argument, much as with the `fit` method. It’s important to note that this argument is allowed to be a data generator, but it could also be a `tuple of Numpy arrays`. If you pass a generator as validation_data , then this generator is expected to yield batches of validation data endlessly; thus you should also specify the `validation_steps` argument, which tells the process how many batches to draw from the validation generator for evaluation.

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50)

One can also save her models after training.

In [ ]:
model.save('cats_and_dogs_small_1.h5')

In [ ]:
model

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()